In [ ]:
import os
import time
import json
import numpy as np
import pandas as pd
from scipy import stats
import random
from dotenv import load_dotenv
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

In [ ]:
# Cargar variables de entorno
load_dotenv()
client = MistralClient(api_key="XEV0fCx3MqiG9HqVkGc4Hy5qyD3WwPHr")
corpus_path='../data/processed/normalized_data.json'

In [ ]:
# 1. Cargar el corpus de conocimiento
def load_corpus(corpus_path='../data/processed/normalized_data.json'):
    """Carga el corpus de conocimiento desde un archivo JSON"""
    with open(corpus_path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [ ]:

# 2. Generar preguntas basadas en el corpus
def generate_corpus_questions(corpus, n=30):
    """Genera preguntas relevantes basadas en el corpus usando Mistral AI"""
    # Extraer información clave del corpus para construir un contexto
    cities = set()
    attractions = set()
    unique_titles = set()
    
    for entry in corpus:
        cities.add(entry['city'])
        if entry.get('attractions'):
            attractions.update(entry['attractions'])
        if entry.get('title'):
            # Limpiar título eliminando caracteres especiales
            title = entry['title'].replace('\r', '').replace('\n', '').replace('\t', '').strip()
            if title and len(title) < 100:  # Filtrar títulos muy largos
                unique_titles.add(title)
    
    # Construir contexto para el prompt
    context = f"""
    Estás generando preguntas turísticas sobre Cuba basadas en este contexto:
    - Ciudades mencionadas: {', '.join(cities)}
    - Atracciones turísticas: {', '.join(attractions) if attractions else 'No especificadas'}
    - Lugares destacados: {', '.join(unique_titles) if unique_titles else 'No especificados'}
    
    Genera preguntas que:
    1. Sean relevantes para la información en el corpus
    2. Cubran diferentes aspectos del turismo en Cuba
    3. Sean específicas pero naturales (como las haría un turista)
    4. Incluyan referencias a ciudades, atracciones y conceptos del contexto
    5. Referentes a estos temas solamente: gastronomía, vida nocturna, 
    lugares históricos y alojamiento
    """
    
    # Crear prompt para Mistral AI
    prompt = f"""
    {context}
    
    Genera EXACTAMENTE {n} preguntas sobre turismo en Cuba usando este formato:
    [PREGUNTA 1]
    [PREGUNTA 2]
    ...
    [PREGUNTA {n}]
    
    Reglas:
    - Solo incluye la pregunta sin numeración
    - Usa diferentes tipos de preguntas (qué, dónde, cómo)
    - Refiérete específicamente a: {', '.join(list(cities)[:5])}...
    """
    
    try:
        print("generando preguntas...")
        # Obtener respuesta de Mistral AI
        response = client.chat(
            model="mistral-large-latest",
            messages=[{"role": "user", "content": prompt}]
        )
        
        print("preguntas generadas...")
        
        # Procesar las preguntas generadas
        content = response.choices[0].message.content
        questions = []
        
        for line in content.split('\n'):
            clean_line = line.strip()
            if clean_line and clean_line.endswith('?'):
                questions.append(clean_line)
        
        # Asegurarnos de tener exactamente n preguntas
        if len(questions) >= n:
            return questions[:n]
        else:
            # Generar preguntas de respaldo si no hay suficientes
            base_questions = []
            return base_questions[:n]
            
    except Exception as e:
        print(f"Error generando preguntas: {str(e)}")

In [ ]:
# Verificar si ya existen preguntas
try:
    existing_questions = pd.read_csv('preguntas_evaluacion.csv')
    if not existing_questions.empty:
        questions = existing_questions['pregunta'].tolist()
        print(f"Se encontraron {len(questions)} preguntas existentes en preguntas_evaluacion.csv")
    else:
        raise FileNotFoundError
except:
    print("Generando nuevas preguntas...")
    corpus = load_corpus(corpus_path)
    questions = generate_corpus_questions(corpus, 30)
    pd.DataFrame({'pregunta': questions}).to_csv('questions.csv', index=False)
    print(f"Se generaron {len(questions)} preguntas nuevas y se guardaron en preguntas_evaluacion.csv")
    